In [1]:
!pip install supervision
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.7 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from ultralytics import YOLO
import cv2
import numpy as np
import json

# ============================
# 🚀 Khởi tạo mô hình và video
# ============================

# Tải mô hình YOLO đã huấn luyện
model_pitch = YOLO("/content/drive/MyDrive/Thử nghiệm tối ưu/best_n.pt")

# Đường dẫn video đầu vào
video_path = "/content/drive/MyDrive/Thử nghiệm tối ưu/Test30s.mp4"
cap = cv2.VideoCapture(video_path)

# Thiết lập video đầu ra
save_output = True
if save_output:
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter("output_full_prediction_fs16.mp4", fourcc, fps, (width, height))

# ====================================
# 🎨 Hàm vẽ bounding box và keypoints
# ====================================
def draw_detections(frame, boxes_with_conf, labels, keypoints=None):
    """
    Vẽ bounding box, nhãn và keypoints lên khung hình.

    Args:
        frame (np.ndarray): Khung hình gốc.
        boxes_with_conf (list): Danh sách bounding box kèm độ tin cậy.
        labels (list): Danh sách nhãn tương ứng.
        keypoints (list, optional): Danh sách tọa độ keypoints.

    Returns:
        np.ndarray: Khung hình đã vẽ.
    """
    color_map = {'pitch': (0, 255, 0)}

    for box, label in zip(boxes_with_conf, labels):
        x1, y1, x2, y2, conf_val = (box + [0.0])[:5]
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
        color = color_map.get(label, (200, 200, 200))

        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, f"{label} {conf_val:.2f}", (x1, max(y1 - 10, 15)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    if keypoints:
        for idx, (x, y) in enumerate(keypoints):
            if x > 0 and y > 0:
                x, y = map(int, [x, y])
                cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)
                cv2.putText(frame, str(idx + 1), (x + 6, y - 6),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    return frame

# ==========================================
# 🔍 Dự đoán từng khung hình và lưu kết quả
# ==========================================
full_predictions = []
frame_idx = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Dự đoán bằng mô hình YOLO
    results = model_pitch(frame, half=True)[0]

    boxes, labels, confs = [], [], []

    for b in results.boxes:
        box = [float(x) for x in b.xyxy[0].cpu().numpy()]
        conf = float(b.conf[0])
        label = results.names[int(b.cls[0])]

        boxes.append(box + [conf])
        labels.append(label)
        confs.append(conf)

    # Trích xuất keypoints nếu có
    keypoints = []
    if hasattr(results, "keypoints") and results.keypoints and len(results.keypoints.xy) > 0:
        keypoints = [[float(x), float(y)] for x, y in results.keypoints.xy[0].cpu().numpy()]

    # Lưu dữ liệu khung hình vào danh sách
    frame_data = {
        "frame_id": frame_idx,
        "pitch": {
            "boxes": [
                {"bbox": box[:4], "conf": conf, "class": label}
                for box, conf, label in zip(boxes, confs, labels)
            ],
            "keypoints": keypoints
        }
    }
    full_predictions.append(frame_data)

    # Vẽ và ghi khung hình nếu cần
    frame_drawn = draw_detections(frame, boxes, labels, keypoints)
    if save_output:
        out.write(frame_drawn)

    frame_idx += 1

# ============================
# 💾 Lưu video và file JSON
# ============================
cap.release()
if save_output:
    out.release()

with open("predictions_full_no_interpolation.json", "w") as f:
    json.dump(full_predictions, f, indent=4)

print(f"✅ Đã xử lý {frame_idx} khung hình. Kết quả lưu tại 'predictions_full_no_interpolation.json'.")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.

0: 384x640 1 pitch, 3616.3ms
Speed: 22.0ms preprocess, 3616.3ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 10.1ms
Speed: 2.2ms preprocess, 10.1ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 10.7ms
Speed: 2.0ms preprocess, 10.7ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 11.9ms
Speed: 1.9ms preprocess, 11.9ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 10.5ms
Speed: 1.9ms preprocess, 10.5ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 11.1ms
Speed: 1.9ms preprocess, 11.1ms